# 4 - Mar - 2019
## Session: Third generation data assembly
### By Luis AF. 

#### Stolen slides:
https://www.slideshare.net/agbiotec/overview-of-genome-assembly-algorithms

# De Brujin Graphs.

## Minisuite

In [8]:
!minimap2 -x ava-ont Lambda.fastq Lambda.fastq > Lambda.paf
!miniasm -f Lambda.fastq Lambda.paf > Lambda.gfa
!awk '/^S/{print ">"$2"\n"$3}' Lambda.gfa | fold > Lambda-minimap2.fa

[M::mm_idx_gen::1.721*1.33] collected minimizers
[M::mm_idx_gen::2.339*1.77] sorted minimizers
[M::main::2.339*1.77] loaded/built the index for 16332 target sequence(s)
[M::mm_mapopt_update::2.486*1.72] mid_occ = 493
[M::mm_idx_stat] kmer size: 15; skip: 5; is_hpc: 0; #seq: 16332
[M::mm_idx_stat::2.570*1.70] distinct minimizers: 11951208 (78.43% are singletons); average occurrences: 2.377; average spacing: 2.947
[M::worker_pipeline::107.467*2.42] mapped 16332 sequences
[M::main] Version: 2.16-r922
[M::main] CMD: minimap2 -x ava-ont Lambda.fastq Lambda.fastq
[M::main] Real time: 107.492 sec; CPU: 260.000 sec; Peak RSS: 2.293 GB
[M::main] ===> Step 1: reading read mappings <===
[M::ma_hit_read::13.557*0.87] read 15995233 hits; stored 18830026 hits and 10802 sequences (76770717 bp)
[M::main] ===> Step 2: 1-pass (crude) read selection <===
[M::ma_hit_sub::15.789*0.89] 10742 query sequences remain after sub
[M::ma_hit_cut::15.981*0.89] 18799858 hits remain after cut
[M::ma_hit_flt::16.217*0

## wtbg2

In [14]:
!wtdbg2 -x rs -g 48k -i Lambda.fastq -t4 -fo prefix
# derive consensus
!wtpoa-cns -t4 -i prefix.ctg.lay.gz -fo prefix.ctg.fa
# polish consensus, not necessary if you want to polish the assemblies using other tools
#minimap2 -t 4 -x map-ont -a prefix.ctg.fa reads.fa.gz
#samtools view -Sb - >prefix.ctg.map.bam
#samtools sort prefix.ctg.map.bam prefix.ctg.map.srt
#samtools view prefix.ctg.map.srt.bam 
#wtpoa-cns -t 16 -d prefix.ctg.fa -i - -fo prefix.ctg.2nd.fa

--
-- total memory       16350888.0 kB
-- available           7251352.0 kB
-- 8 cores
-- Starting program: wtdbg2 -x rs -g 48k -i Lambda.fastq -t4 -fo prefix
-- pid                     26444
-- date         Mon Mar  4 13:10:43 2019
--
[Mon Mar  4 13:10:43 2019] loading reads
5605 reads
[Mon Mar  4 13:10:43 2019] filtering from 5605 reads (>=5000 bp), 59268629 bp. Try selecting 2400000 bp
[Mon Mar  4 13:10:43 2019] Done, 60 reads (>=5000 bp), 2427904 bp, 9354 bins
** PROC_STAT(0) **: real 0.101 sec, user 0.070 sec, sys 0.030 sec, maxrss 52404.0 kB, maxvsize 210084.0 kB
[Mon Mar  4 13:10:43 2019] Set --edge-cov to 3
KEY PARAMETERS: -k 0 -p 21 -K 1000.049988 -S 4.000000 -s 0.050000 -g 48000 -X 50.000000 -e 3 -L 5000
[Mon Mar  4 13:10:43 2019] generating nodes, 4 threads
[Mon Mar  4 13:10:43 2019] indexing bins[0,9354] (2394624 bp), 4 threads
[Mon Mar  4 13:10:43 2019] - scanning kmers (K0P21S4.00) from 9354 bins
9354 bins
********************** Kmer Frequency **********************
|     

## Flye

In [19]:
!flye --nano-raw Lambda.fastq --out-dir Lambda-Flye --genome-size 48k  --threads 4 --asm-coverage 100

[2019-03-04 13:30:30] INFO: Starting Flye 2.4-release
[2019-03-04 13:30:30] INFO: >>>STAGE: configure
[2019-03-04 13:30:30] INFO: Configuring run
[2019-03-04 13:30:30] INFO: Input genome size: 48000
[2019-03-04 13:30:30] INFO: Estimated coverage: 1743
[2019-03-04 13:30:30] WARNING: Expected read coverage is 1743, the assembly is not guaranteed to be optimal in this setting. Are you sure that the genome size was entered correctly?
[2019-03-04 13:30:30] INFO: Reads N50/N90: 8327 / 2385
[2019-03-04 13:30:30] INFO: Minimum overlap set to 2000
[2019-03-04 13:30:30] INFO: Selected k-mer size: 15
[2019-03-04 13:30:30] INFO: Using longest 100x reads for contig assembly
[2019-03-04 13:30:30] INFO: >>>STAGE: assembly
[2019-03-04 13:30:30] INFO: Assembling reads
[2019-03-04 13:30:30] INFO: Reading sequences
[2019-03-04 13:30:31] INFO: Generating solid k-mer index
[2019-03-04 13:30:45] INFO: Counting kmers (1/2):
0% 10% 20% 30% 40% 50% 60% 70% 80% 90% 100% 
[2019-03-04 13:30:45] INFO: Counting kme

# OLC assemblers

In [ ]:
!canu -p Lambda-4March19 -d Lambda-1 genomeSize=48k -nanopore-raw Lambda.fastq

In [27]:
!canu -p Lambda-4March19-tuned -d Lambda-2 genomeSize=48k corOutCoverage=1000 corErrorRate=0.12 -nanopore-raw Lambda.fastq 

In [24]:
!/usr/lib/abyss/abyss-fac Lambda-minisuite.fa
!/usr/lib/abyss/abyss-fac Lambda-wtdbg2.fa
!/usr/lib/abyss/abyss-fac Lambda-Flye.fasta

n	n:500	L50	min	N80	N50	N20	E-size	max	sum	name
1	1	1	49883	49883	49883	49883	49883	49883	49883	Lambda-minisuite.fa
n	n:500	L50	min	N80	N50	N20	E-size	max	sum	name
1	1	1	48154	48154	48154	48154	48154	48154	48154	Lambda-wtdbg2.fa
n	n:500	L50	min	N80	N50	N20	E-size	max	sum	name
1	1	1	85693	85693	85693	85693	85693	85693	85693	Lambda-Flye.fasta


#  Homework

1.
https://github.com/HingeAssembler/HINGE

2. 
https://github.com/rrwick/Unicycler

3.
Assembly metrics:
a. What the kwanza is a N50? 
b. Tools for assembly metrics?
c. BUSCO?